In [20]:
#import statements
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from pathlib import Path
import os
from sklearn.model_selection import train_test_split
from helper_methods import print_schema, encode_numeric_zscore, encode_text_dummy, print_column

In [21]:
#read from train/test datasets
training_dataframe = pd.read_csv(os.path.join(os.getcwd(), 'data/UNSW_NB15_training-set.csv'))
testing_dataframe = pd.read_csv(os.path.join(os.getcwd(), 'data/UNSW_NB15_testing-set.csv'))

In [22]:
#drop all entries that have a missing value anywhere in the set
training_dataframe = training_dataframe.replace('-', np.nan).dropna()
testing_dataframe = testing_dataframe.replace('-', np.nan).dropna()


In [23]:
testing_dataframe.shape

(35179, 45)

In [24]:
#get one instance of each value from each dataframe
training_unique = training_dataframe["proto"].unique()
testing_unique = testing_dataframe["proto"].unique()

#get only values from each numpy array
common_values = np.intersect1d(training_unique, testing_unique)

#make a filtered dataframe containing only values in common_values
training_dataframe = training_dataframe[training_dataframe['proto'].isin(common_values)]
testing_dataframe = testing_dataframe[testing_dataframe['proto'].isin(common_values)]

#same for service
training_unique = training_dataframe["service"].unique()
testing_unique = testing_dataframe["service"].unique()
common_values = np.intersect1d(training_unique, testing_unique)
training_dataframe = training_dataframe[training_dataframe['service'].isin(common_values)]
testing_dataframe = testing_dataframe[testing_dataframe['service'].isin(common_values)]

#same for state
training_unique = training_dataframe["state"].unique()
testing_unique = testing_dataframe["state"].unique()
common_values = np.intersect1d(training_unique, testing_unique)
training_dataframe = training_dataframe[training_dataframe['state'].isin(common_values)]
testing_dataframe = testing_dataframe[testing_dataframe['state'].isin(common_values)]


In [25]:
testing_dataframe.shape

(35178, 45)

In [26]:
print_schema(training_dataframe)
print_schema(testing_dataframe)

~~~~~~dataframe schema~~~~~~
Dataframe shape: (81159, 45) | Dataframe length: 81159
Column labels: 
Index(['id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label'],
      dtype='object')
Dataframe head: 
    id       dur proto service state  spkts  dpkts  sbytes  dbytes  \
3    4  1.681642   tcp     ftp   FIN     12     12     628     770   
11  12  2.093085   tcp    smtp   FIN     62     28   56329    2212   
15  16  0.000002   udp    snmp   INT      2      0     138       0   
17  18  

### Encode Data

One-hot encode relevant rows


In [27]:
# one-hot encode non-y features of training dataset
encode_text_dummy(training_dataframe, 'proto')
encode_text_dummy(training_dataframe, 'service')
encode_text_dummy(training_dataframe, 'state')

# one-hot encode attack-category column (y)
encode_text_dummy(training_dataframe, 'attack_cat')

# one-hot encode non-y features of testing dataset
encode_text_dummy(testing_dataframe, 'proto')
encode_text_dummy(testing_dataframe, 'service')
encode_text_dummy(testing_dataframe, 'state')

# one-hot encode attack-category column (y)
encode_text_dummy(testing_dataframe, 'attack_cat')


In [28]:
print_schema(training_dataframe)
print_schema(testing_dataframe)


~~~~~~dataframe schema~~~~~~
Dataframe shape: (81159, 68) | Dataframe length: 81159
Column labels: 
Index(['id', 'dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl',
       'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit',
       'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat',
       'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src',
       'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd',
       'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'label', 'proto-tcp',
       'proto-udp', 'service-dhcp', 'service-dns', 'service-ftp',
       'service-ftp-data', 'service-http', 'service-irc', 'service-pop3',
       'service-radius', 'service-smtp', 'service-snmp', 'service-ssh',
       'service-ssl', 'state-CON', 'state-FIN', 'state-INT', 'state-REQ',
       'attack_cat-Analysis', 'attack_cat-Backdoor', 'attack_cat-DoS',
       '

Z-Score relevant rows

In [29]:
encode_numeric_zscore(training_dataframe, 'dur')
encode_numeric_zscore(training_dataframe, 'spkts')
encode_numeric_zscore(training_dataframe, 'dpkts')
encode_numeric_zscore(training_dataframe, 'sbytes')
encode_numeric_zscore(training_dataframe, 'dbytes')
encode_numeric_zscore(training_dataframe, 'rate')
encode_numeric_zscore(training_dataframe, 'sttl')
encode_numeric_zscore(training_dataframe, 'dttl')
encode_numeric_zscore(training_dataframe, 'sload')

encode_numeric_zscore(testing_dataframe, 'dur')
encode_numeric_zscore(testing_dataframe, 'spkts')
encode_numeric_zscore(testing_dataframe, 'dpkts')
encode_numeric_zscore(testing_dataframe, 'sbytes')
encode_numeric_zscore(testing_dataframe, 'dbytes')
encode_numeric_zscore(testing_dataframe, 'rate')
encode_numeric_zscore(testing_dataframe, 'sttl')
encode_numeric_zscore(testing_dataframe, 'dttl')
encode_numeric_zscore(testing_dataframe, 'sload')

In [30]:
print_schema(training_dataframe)
print_schema(testing_dataframe)

~~~~~~dataframe schema~~~~~~
Dataframe shape: (81159, 68) | Dataframe length: 81159
Column labels: 
Index(['id', 'dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl',
       'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit',
       'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat',
       'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src',
       'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd',
       'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'label', 'proto-tcp',
       'proto-udp', 'service-dhcp', 'service-dns', 'service-ftp',
       'service-ftp-data', 'service-http', 'service-irc', 'service-pop3',
       'service-radius', 'service-smtp', 'service-snmp', 'service-ssh',
       'service-ssl', 'state-CON', 'state-FIN', 'state-INT', 'state-REQ',
       'attack_cat-Analysis', 'attack_cat-Backdoor', 'attack_cat-DoS',
       '